In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from teste_datas import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot

from bokeh import palettes

In [2]:
# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [3]:
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'
fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

In [4]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)
    
counts = spec2.spCounts
print(len(counts))

ret = cwt_complete_analysis(counts)
# ret = gaussian_complete_analysis(counts)

(xs, y0s, ar1, lblimag1, num_feat_imag1, ridges, xpks, ypks,
 ind_ypks, retang_indicess, peaks_masks, cwtpks, nscpks) = ret

8192
8192


In [5]:
# 2020-05-15 Espectros Mariana PR.

In [6]:
type(spec2)

teste_datas.Spec

In [7]:
# Desativado:

In [8]:
(nscpk_ma, cwtpk_ma, pospk_ma) = peaks_masks
valid_peaks = ~(cwtpk_ma.mask | nscpk_ma.mask | ~pospk_ma)

In [9]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# Bokeh graph w/ peaks in all scales

# Gráfico de ridges com qualidade dos picos pelas cores e tamanhos:

# Navy 2:     pontos dos ridgest
# Vermelho 8: picos válidos
# Laranja 8:  inválido, aparece em poucas escalas  
# Verde 6:    inválido, muito pequeno (pelo coeficiente cwt)
# Ciano 3:    inválido, forma ruim (cwt perto dos extremos do ridge)

bokridges1 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   # x_axis_label='channels',
   y_axis_label='scale',
   title=fn2,
)
bokridges1.circle(ar1[:,0], ar1[:,1], color="navy",size=2)
bokridges1.circle(xpks[valid_peaks], ypks[valid_peaks], color="red",size=8)
bokridges1.circle(xpks[nscpk_ma.mask], ypks[nscpk_ma.mask], color="orange",size=8)
bokridges1.circle(xpks[cwtpk_ma.mask], ypks[cwtpk_ma.mask], color="green",size=6)
bokridges1.circle(xpks[~pospk_ma], ypks[~pospk_ma], color="cyan",size=3)
bokridges2 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", y_range=[1.0e1, 1.0e5],
   x_axis_label='channels', y_axis_label='counts'
)
bokridges2.line(xs, y0s, color="green")
# https://docs.bokeh.org/en/latest/docs/reference/layouts.html
p = gridplot([bokridges1, bokridges2], ncols=1)
show(p)

In [10]:
valid_xpks = np.asarray(xpks)[valid_peaks]
valid_xpks

array([ 113,  158,  236,  321,  391,  427,  442,  470,  709,  729,  828,
        941, 1005, 1209, 1713, 1783, 2587, 2953, 3084, 4067, 7403,  827])

In [11]:
valid_ypks = np.asarray(ypks)[valid_peaks]
valid_ypks

array([ 7.06808163,  4.30367347,  6.14661224,  3.84293878,  2.92146939,
        4.30367347,  2.92146939,  5.22514286,  5.68587755,  4.76440816,
        2.92146939,  4.76440816,  5.22514286,  3.84293878,  8.91102041,
        5.22514286, 12.59689796,  5.22514286,  4.76440816,  6.60734694,
       11.21469388,  6.60734694])

In [12]:
valid_cwtpks = np.asarray(cwtpks)[valid_peaks]
valid_cwtpks

array([363.92344023, 157.66410398, 132.62170779, 278.8749593 ,
        52.74309673,  78.8691267 ,  63.1045599 , 626.81827194,
        64.04584194, 109.11190158,  40.23306285, 408.37033302,
        63.96726554, 151.77446391,  35.54886839,  65.1724873 ,
       689.55861672,  44.67953049,  59.55391228,  37.30707438,
       111.10264647,  40.84078107])

In [13]:
xs

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 8.189e+03, 8.190e+03,
       8.191e+03])

In [14]:
y0s

array([0, 0, 0, ..., 6, 5, 8])

In [15]:
step2 = cwt_calc_peaks(valid_xpks, valid_ypks, valid_cwtpks, xs, y0s)

In [16]:
xpks[pospk_ma]


array([ 113,  158,  236,  321,  391,  427,  442,  470,  709,  729,  828,
        941, 1005, 1209, 1348, 1479, 1512, 1679, 1713, 1743, 1783, 1985,
       2027, 2311, 2587, 2812, 2953, 3019, 3063, 3084, 3103, 3240, 3297,
       3350, 3489, 3513, 3678, 3864, 3884, 3926, 3943, 4067, 4231, 4244,
       4260, 4353, 4524, 4668, 4708, 4729, 4876, 4911, 4954, 5072, 5091,
       5348, 5382, 5600, 5677, 5704, 5749, 5976, 6046, 6065, 6203, 6269,
       6358, 6414, 6457, 6576, 6689, 6743, 6762, 6874, 7046, 7180, 7246,
       7403, 7542, 7756, 7956, 7972, 8032, 8071, 1413, 1493, 1801, 2825,
       5953, 6618, 7077, 1583, 2112, 4614, 5170, 5197, 5322, 5364, 5828,
       6002, 7505, 8182, 1818, 2398, 4464, 5459, 6320, 6492, 7029, 2293,
       3710, 4001, 6785, 6806, 7652, 1924, 3388, 7162, 7310, 7829, 4827,
       6907, 7993, 5578, 7606,  827, 3420, 3824, 7856, 2230, 4313, 4121,
       5202, 6075, 8144, 2285, 5802, 7116, 6656, 6455, 4962, 4736, 5868,
       6921, 2107])

In [17]:
(azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
           baseline_spectrum, bl_median_smoother) = step2

In [18]:
azes

[array([108., 109., 110., 111., 112., 113., 114., 115., 116., 117., 118.]),
 array([153., 154., 155., 156., 157., 158., 159., 160., 161., 162., 163.]),
 array([231., 232., 233., 234., 235., 236., 237., 238., 239., 240., 241.]),
 array([316., 317., 318., 319., 320., 321., 322., 323., 324., 325., 326.]),
 array([386., 387., 388., 389., 390., 391., 392., 393., 394., 395., 396.]),
 array([422., 423., 424., 425., 426., 427., 428., 429., 430., 431., 432.]),
 array([437., 438., 439., 440., 441., 442., 443., 444., 445., 446., 447.]),
 array([465., 466., 467., 468., 469., 470., 471., 472., 473., 474., 475.]),
 array([704., 705., 706., 707., 708., 709., 710., 711., 712., 713., 714.]),
 array([724., 725., 726., 727., 728., 729., 730., 731., 732., 733., 734.]),
 array([823., 824., 825., 826., 827., 828., 829., 830., 831., 832., 833.]),
 array([936., 937., 938., 939., 940., 941., 942., 943., 944., 945., 946.]),
 array([1000., 1001., 1002., 1003., 1004., 1005., 1006., 1007., 1008.,
        1009., 10

In [19]:
len(azes)

22

In [20]:
bezes

[array([  0. , -27.7, -24.4, -57.1, -10.8, -20.5,  -9.2,  -7.9,  -1.6,
         34.7,   0. ]),
 array([  0.,   1.,   4.,  48.,  78.,  70.,  61.,  21., -11.,  -9.,   0.]),
 array([  0. , -19.6,   7.8,  33.2,  42.6,  63. ,  66.4,  44.8,  -3.8,
         -7.4,   0. ]),
 array([  0. ,   9.7,  43.4, 118.1, 185.8, 157.5, 166.2,  93.9,  30.6,
        -10.7,   0. ]),
 array([ 0. , 10.6, 31.2, 32.8, 26.4, 55. , 22.6,  8.2, -8.2, -6.6,  0. ]),
 array([ 0. ,  6.7, 21.4, 17.1, 38.8, 49.5, 29.2,  8.9, 32.6, 17.3,  0. ]),
 array([ 0. , 34.6, 18.2, 29.8, 59.4, 46. , 49.6, 41.2, 27.8,  6.4,  0. ]),
 array([  0. ,  72.9, 193.8, 282.7, 355.6, 320.5, 230.4, 159.3,  89.2,
         57.1,   0. ]),
 array([  0. ,   2.7,  25.4,  21.1,  29.8,  19.5,  41.2,   8.9, -18.4,
        -12.7,   0. ]),
 array([ 0. ,  2.1, 12.2, 22.3, 58.4, 38.5, 65.6, 35.7, 19.8, 20.9,  0. ]),
 array([  0. , -12.3,  16.4,   7.1,  50.8,  21.5,  37.2,  23.9,  26.6,
         11.3,   0. ]),
 array([  0. ,  41.9, 100.8, 192.7, 213.6, 229.5, 

In [21]:
centrpk

[109.64257028112449,
 157.40684410646386,
 236.26872246696036,
 320.76337319068585,
 389.6046511627907,
 427.2257336343115,
 441.77316293929715,
 469.5747942094805,
 707.2468085106383,
 729.4791288566244,
 828.7945205479452,
 940.573612931411,
 1004.9869706840391,
 1208.286384976526,
 1716.64,
 1783.7155172413795,
 2586.9029982363318,
 2953.8555555555554,
 3083.89898989899,
 4066.161572052402,
 7402.728045325779,
 828.0129032258066]

In [22]:
#azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
#           baseline_spectrum, bl_median_smoother) = step2

In [23]:
bezes[0]

array([  0. , -27.7, -24.4, -57.1, -10.8, -20.5,  -9.2,  -7.9,  -1.6,
        34.7,   0. ])

In [24]:
gau_areas

[1466.3943432140982,
 495.7268616601767,
 498.33626563213693,
 828.5738478158929,
 136.63320763632893,
 247.98000099811637,
 163.47501323258086,
 2171.6043407634265,
 231.46197765033014,
 360.9662299956111,
 104.22543937058417,
 1350.9791087691829,
 221.61381971936092,
 450.9417119270606,
 160.83459460182027,
 225.78929596758425,
 3709.3085826824713,
 154.7916943490386,
 197.01747368987273,
 145.34319994067187,
 563.9077776403681,
 159.1100322966713]

In [25]:
# 2020-02-24
# Tudo em progresso! Agora:
# - Fazer os gr'aficos
# - fazer em v'arias passadas o c'alculo at'e n~ao haver picos na linha-base.

In [26]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   y_axis_type="log",
   y_range=[10**1, 10**5],
   title=fn2,
   # y_range=[-1000.0, 1000.0],
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
bokpksallscl.line(xs, y0s, color="green")
bokpksallscl.line(xs, peak_constructed_spectrum, color="red")
bokpksallscl.line(xs, bl_median_smoother, color="orange")
bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [27]:
# Bokeh graph w/ peaks in all scales
bok2 = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5],
   title=fn2,
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="green")
bok2.line(xs, peak_constructed_spectrum, color="red")
# bokpksallscl.line(xs, bl_median_smoother, color="cyan")
# bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bok2)

In [28]:
# 2020-02-25 Segunda passada:
# 2020-05-15 Por enquanto deixarei desativada; mas posso fazer manualmente, bastando rodar
#            as duas células seguintes e repetir o trecho que se inicia logo após
#            a determinação de valid_peaks.

In [29]:
# 2020-05-17 Continuando para cada espectro. Este é agua-bruta.iec

In [30]:
spec2.samDesc

'Agua bruta; m=109,0g - sel 16/10/19; ; .'

In [31]:
print(spec2.spLVTime)

250000.0


In [32]:
print(spec2.spRLTime)

250198.53


In [33]:
print(spec2.diahora)

2019-12-20 15:05:14


In [34]:
aiai = 'aiuh'

In [35]:
aiai = aiai[:-2]+'~'

In [36]:
aiai

'ai~'

In [37]:
import pickle

In [38]:
len(counts)

8192

In [39]:
dmp_counts = pickle.dumps(counts)
type(dmp_counts)

bytes

In [40]:
len(dmp_counts)

16456

In [41]:
ldd_counts = pickle.loads(dmp_counts)
print(type(ldd_counts))
len(ldd_counts)

<class 'list'>


8192

In [42]:
print(dmp_counts)

b'\x80\x03]q\x00(K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00K\x00M6\x01M6\x01M\x13\x01M\x0f\x01K\xe7M\x0e\x01K\xfdM\x01\x01K\xfbK\xfaM\x17\x01K\xedM\x02\x01M\x17\x01K\xf0K\xf0M\x0e\x01K\xf0K\xe3K\xf8K\xf2M\'\x01K\xfdK\xfaM\x16\x01M\x0c\x01M\x00\x01K\xf2M*\x01MN\x01M\xd2\x01M@\x02M\x93\x02Mn\x02M\xc5\x01Mo\x01K\xf5K\xf6K\xcbK\xc3K\xbdK\xb1K\xc7K\xd2K\xbfK\xceK\xcaK\xccK\xd0K\xfdM\x1c\x01M\x15\x01M\r\x01K\xe6K\xc7K\xcaK\xd4K\xa1K\xabK\x99K\xa4K\xa0K\x88K\x94K\x9eK\x97K\xa0K\x9aK\x98K\x9cK\x93K\x97K\xb8K\x9fK\x

In [43]:
type(counts[-1])

int

In [44]:
16456/8

2057.0

In [45]:
ai = 7
type(ai)

int

In [46]:
ai2 = 256*256
type(ai2)

int

In [47]:
ai3 = ai2*ai2
type(ai3)

int

In [48]:
ai4 = ai3*ai3*ai3*ai3*ai3*ai3
type(ai4)

int

In [49]:
ai4

6277101735386680763835789423207666416102355444464034512896